In [1]:
install.packages(c("tidyverse", "readtext", 
    "quanteda", "quanteda.textmodels", 
    "topicmodels", "keras"))

In [1]:
# General packages and dictionary analysis
library(glue)
library(tidyverse)
library(readtext)
library(quanteda)

# Supervised text classification
library(quanteda.textmodels)
library(MLmetrics)

# Topic Modeling
library(topicmodels)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::collapse() masks glue::collapse()
✖ dplyr::filter()   masks stats::filter()
✖ dplyr::lag()      masks stats::lag()

Package version: 3.0.0
Unicode version: 13.0
ICU version: 66.1

Parallel computing: 8 of 8 threads used.

See https://quanteda.io for tutorials and examples.



ERROR: Error in library(quanteda.textmodels): there is no package called ‘quanteda.textmodels’


In [2]:
filename = "reviewdata.rds"
if (file.exists(filename)) {
    print("Using cached data")
    reviewdata= readRDS(filename)
} else {
    print("Downloading data")
    download.file("https://cssbook.net/d/aclImdb_v1.tar.gz", "aclImdb_v1.tar.gz")
    untar("aclImdb_v1.tar.gz")
    reviewdata = readtext(file.path("aclImdb", "*", "*", "*.txt"), 
                          docvarsfrom = "filepaths", dvsep="[/\\]",
                          docvarnames=c("imdb", "dataset", "label", "filename"))
    unlink(c("aclImdb", "aclImdb_v1.tar.gz"), recursive=TRUE)
    reviewdata = reviewdata  %>% filter(label %in% c("pos", "neg")) %>% select(-imdb)  %>% corpus()
    saveRDS(reviewdata, filename)
}
head(docvars(reviewdata))

[1] "Downloading data"


,dataset,label,filename
,<chr>,<chr>,<chr>
1,test,neg,0_2.txt
2,test,neg,1_3.txt
3,test,neg,10_3.txt
4,test,neg,100_4.txt
5,test,neg,1000_3.txt
6,test,neg,10000_4.txt


In [7]:
positive = scan("https://cssbook.net/d/positive.txt", what="list")
negative = scan("https://cssbook.net/d/negative.txt", what="list")
mydict = dictionary(list(pos=positive, neg=negative))


# let"s select only the first 100 reviews for now to speed things up

scores = corpus_sample(reviewdata, 100)  %>% 
  dfm(dictionary=mydict) %>% 
  convert(to="data.frame")  %>% 
  mutate(sent = pos - neg)
head(scores)

,doc_id,pos,neg,sent
,<chr>,<dbl>,<dbl>,<dbl>
1,test/pos/5960_8.txt/5960_8.txt,8,3,5
2,train/neg/2622_4.txt/2622_4.txt,4,2,2
3,train/neg/4605_2.txt/4605_2.txt,20,12,8
4,test/pos/4487_9.txt/4487_9.txt,3,2,1
5,test/neg/7566_1.txt/7566_1.txt,3,4,-1
6,test/neg/11817_2.txt/11817_2.txt,8,8,0
